<a href="https://colab.research.google.com/github/defneyucesir/gametheory/blob/main/Sentiment_Analysis_and_Defining_Utilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
!pip install spacy
!pip install scattertext
!pip install tika
!pip install spacytextblob
!python -m spacy download en_core_web_sm
import spacy
import json
import pylab
from IPython.core.display import display, HTML
import nltk
from tika import parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from spacytextblob.spacytextblob import SpacyTextBlob
import requests  # Import requests for URL size checking
import traceback # Import for detailed error messages
import time
import signal

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10., 8.)
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')
import re

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("URL processing timed out")

def process_urls(csv, timeout_seconds=60):
    try:
        df = pd.read_csv(csv, encoding='utf-8-sig')
        # Rename the 'URL' column to 'url' if it exists
        if 'URL' in df.columns:
            df = df.rename(columns={'URL': 'url'})  # Rename the column here
    except:
        print(f"Error reading the file")
        return None

    # Error handling
    if 'url' not in df.columns:
        print("Error: 'url' column is required.")
        return None

    processed_urls = set()
    results = []

    for index, row in df.iterrows():
        url = row['url']  # Get the URL directly

        if url in processed_urls:
            continue

        # Using TIKA parser with timeout
        start_time = time.time()
        try:
            # Set the signal handler and a 60-second alarm
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(timeout_seconds)

            raw = parser.from_file(url)
            text = raw['content']

            # Cancel the alarm if the parsing completes within the timeout
            signal.alarm(0)

            # Data cleaning
            if text:
                text = " ".join(text.split())
                text = re.sub(r"[^a-zA-Z ]+", "", text)
                text = text.lower()

            # Calculate polarity and subjectivity
            if text:
                doc = nlp(text)
                polarity = doc._.blob.polarity
                subjectivity = doc._.blob.subjectivity

                # Classify sentiment based on polarity scores
                if polarity > 0.05:
                    sentiment_score_classification = "Positive"
                elif polarity < -0.05:
                    sentiment_score_classification = "Negative"
                else:
                    sentiment_score_classification = "Neutral"

                # Store results without country
                results.append([url, polarity, subjectivity, sentiment_score_classification])
                processed_urls.add(url)

        # Error handling with detailed traceback
        except TimeoutException:
            print(f"URL processing timed out for: {url}")
            continue  # Skip to the next URL
        except Exception as e:
            print(f"Error processing URL: {url}")  # Print URL for debugging
            print(f"Error Type: {type(e).__name__}") # Print error type
            print(f"Error Message: {e}") # Print error message
            traceback.print_exc() # Print detailed traceback
            continue # Continue to the next URL
        finally:
            end_time = time.time() # Record end time
            processing_time = end_time - start_time
            print(f"Processed URL: {url} in {processing_time:.2f} seconds") # Print processing time

    # Create DataFrame without country column
    new_df = pd.DataFrame(results, columns=['url', 'polarity', 'subjectivity', 'sentiment_score_classification'])
    return new_df

    pd.set_option("display.max_rows", None)


In [4]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    processed_data = process_urls(fn)

print(processed_data)  # Or display(processed_data)

INFO:tika.tika:Retrieving https://digital-strategy.ec.europa.eu/en/policies/regulatory-framework-ai#:~:text=The%20AI%20Act%20is%20the,legal%20framework%20on%20AI%20worldwide. to /tmp/en-policies-regulatory-framework-ai.


Saving Game Theory.csv to Game Theory (20).csv


INFO:tika.tika:Retrieving https://www.gov.uk/government/publications/ai-regulation-a-pro-innovation-approach/white-paper to /tmp/government-publications-ai-regulation-a-pro-innovation-approach-white-paper.


Processed URL: https://digital-strategy.ec.europa.eu/en/policies/regulatory-framework-ai#:~:text=The%20AI%20Act%20is%20the,legal%20framework%20on%20AI%20worldwide. in 0.80 seconds


INFO:tika.tika:Retrieving https://ico.org.uk/media/about-the-ico/consultation-responses/4029424/regulating-ai-the-icos-strategic-approach.pdf to /tmp/media-about-the-ico-consultation-responses-4029424-regulating-ai-the-icos-strategic-approach.pdf.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tika/tika.py", line 778, in getRemoteFile
    urlretrieve(urlOrPath, destPath)
  File "/usr/lib/python3.11/urllib/request.py", line 241, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
                            ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
               

...

INFO:tika.tika:Retrieving https://thefuturesociety.org/workstream/regulatory-sandboxes-in-the-ai-act/ to /tmp/workstream-regulatory-sandboxes-in-the-ai-act.


Processed URL: https://hai-production.s3.amazonaws.com/files/2023-11/AI-Regulatory-Alignment.pdf in 0.89 seconds


INFO:tika.tika:Retrieving https://www.rand.org/pubs/external_publications/EP70704.html to /tmp/pubs-external_publications-ep70704.html.


Processed URL: https://thefuturesociety.org/workstream/regulatory-sandboxes-in-the-ai-act/ in 0.47 seconds
Error processing URL: https://www.rand.org/pubs/external_publications/EP70704.html
Error Type: HTTPError
Error Message: HTTP Error 403: Forbidden
Processed URL: https://www.rand.org/pubs/external_publications/EP70704.html in 0.21 seconds
                                                   url  polarity  \
0    https://digital-strategy.ec.europa.eu/en/polic...  0.044720   
1    https://www.gov.uk/government/publications/ai-...  0.131598   
2    https://www.deloitte.com/uk/en/Industries/fina...  0.110211   
3    https://www.adalovelaceinstitute.org/report/re...  0.100342   
4    https://www.osborneclarke.com/insights/whats-h...  0.063912   
..                                                 ...       ...   
106  https://www.littler.com/publication-press/publ... -0.600000   
107  https://ibme.ox.ac.uk/wp-content/uploads/Stand...  0.096690   
108                   https://arxiv.org/abs

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tika/tika.py", line 778, in getRemoteFile
    urlretrieve(urlOrPath, destPath)
  File "/usr/lib/python3.11/urllib/request.py", line 241, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
                            ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 563, in error
    return self._call_chain(*args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 496, in _call_chain
    result = func(*

NameError: name 'processed_data' is not defined

In [5]:
from IPython.display import display

average_sentiment = processed_data['polarity'].mean()
print(f"Average Sentiment Score: {average_sentiment}")

total_polarity = processed_data['polarity'].sum()
print(f"Total Polarity Score: {total_polarity}")

print("\nProcessed Data:")
display(processed_data) # Display the DataFrame in a formatted table

def print_polarity_range(percentage):
    polarity_scores = processed_data['polarity']
    lower_percentile = (1 - percentage) / 2
    upper_percentile = 1 - lower_percentile

    lower_bound = polarity_scores.quantile(lower_percentile)
    upper_bound = polarity_scores.quantile(upper_percentile)

    print(f"{percentage*100:.0f}% of polarity scores lie between {lower_bound:.2f} and {upper_bound:.2f}")

# Calculate and print ranges for specified percentages
print_polarity_range(0.70)
print_polarity_range(0.75)
print_polarity_range(0.90)
print_polarity_range(0.95)
print_polarity_range(0.99)

Average Sentiment Score: 0.08249970814664723
Total Polarity Score: 9.157467604277842

Processed Data:


,url,polarity,subjectivity,sentiment_score_classification
0,https://digital-strategy.ec.europa.eu/en/polic...,0.044720,0.404237,Neutral
1,https://www.gov.uk/government/publications/ai-...,0.131598,0.419989,Positive
2,https://www.deloitte.com/uk/en/Industries/fina...,0.110211,0.392379,Positive
3,https://www.adalovelaceinstitute.org/report/re...,0.100342,0.442414,Positive
4,https://www.osborneclarke.com/insights/whats-h...,0.063912,0.333305,Positive
...,...,...,...,...
106,https://www.littler.com/publication-press/publ...,-0.600000,1.000000,Negative
107,https://ibme.ox.ac.uk/wp-content/uploads/Stand...,0.096690,0.395296,Positive
108,https://arxiv.org/abs/2310.04072,0.140794,0.462172,Positive
109,https://hai-production.s3.amazonaws.com/files/...,0.084824,0.439173,Positive


70% of polarity scores lie between 0.04 and 0.13
75% of polarity scores lie between 0.04 and 0.14
90% of polarity scores lie between -0.00 and 0.15
95% of polarity scores lie between -0.03 and 0.16
99% of polarity scores lie between -0.31 and 0.19
